# 03 · Tableau Extracts

**Goal:** Export curated CSVs for Tableau Public dashboard.

In [ ]:
import pathlib
import pandas as pd

DATA = pathlib.Path('../data/processed')
OUT = DATA  # write curated extracts alongside processed data

ae_path = DATA/'monthly_ae.csv'
ecds_path = DATA/'ecds_monthly.csv'
if not ae_path.exists() or not ecds_path.exists():
    print('Processed files not found. Run previous notebooks first.')

if ae_path.exists() and ecds_path.exists():
    ae = pd.read_csv(ae_path, parse_dates=['month_date'])
    ecds = pd.read_csv(ecds_path, parse_dates=['month_date'])

    # Provider monthly KPIs
    ecds_sum = ecds.groupby(['provider_code','provider_name','month_date'], as_index=False)[
        ['type12_attendances','arrivals_12h_or_more']
    ].sum()
    kpi = ae.merge(ecds_sum, on=['provider_code','provider_name','month_date'], how='left')
    kpi['pct_within_4h'] = kpi['within_4h_count'] / kpi['total_attendances']
    kpi['rate_12h_from_arrival'] = kpi['arrivals_12h_or_more'] / kpi['type12_attendances']
    kpi['admit_conversion'] = kpi['emergency_admissions'] / kpi['type12_attendances']
    kpi['over4h_dta_rate'] = kpi['over4h_after_dta'] / kpi['emergency_admissions']

    kpi_out = OUT/'provider_monthly.csv'
    kpi.to_csv(kpi_out, index=False)
    print(f'Exported {kpi_out}')

    # Benchmarks (MoM deltas)
    kpi = kpi.sort_values(['provider_code','month_date'])
    kpi['mom_delta_4h'] = kpi.groupby('provider_code')['pct_within_4h'].diff()
    kpi['mom_delta_12h'] = kpi.groupby('provider_code')['rate_12h_from_arrival'].diff()
    bm_out = OUT/'benchmarks.csv'
    kpi[['provider_code','provider_name','month_date','pct_within_4h','rate_12h_from_arrival','admit_conversion','over4h_dta_rate','mom_delta_4h','mom_delta_12h']].to_csv(bm_out, index=False)
    print(f'Exported {bm_out}')

    # Optional: ECDS age/frailty slice if available
    if 'age_band' in ecds.columns or 'cfs_band' in ecds.columns:
        ecds_slice = ecds.copy()
        ecds_slice_out = OUT/'ecds_age_frailty.csv'
        ecds_slice.to_csv(ecds_slice_out, index=False)
        print(f'Exported {ecds_slice_out}')
